In [2]:
from pyBADA.bada4 import Bada4Aircraft

# 定义所有要查询的飞机型号
aircraft_types = [
    "A320-232",
    "A321-111", 
    "A321-131",
    "A330-203",
    "A330-223",
    "A330-243", 
    "A330-301",
    "A330-321",
    "A330-341",
    "A350-941",
    "A380-841",
    "A380-861",
    "B788GE70",
    "B789GE75",
    "B738W26",
    "B773RR92", 
]

# 文件路径
file_path = "/home/longqin/Downloads/4.2/BADA_4.2_L06514UPC/Models"

print("="*80)
print("BADA 4.2 飞机参数批量获取")
print("="*80)

# 遍历所有飞机型号
for i, ac_name in enumerate(aircraft_types, 1):
    print(f"\n[{i:2d}] 正在处理: {ac_name}")
    print("-" * 50)
    
    try:
        # 创建飞机对象
        AC = Bada4Aircraft(
            badaVersion="4.2",
            acName=ac_name,
            filePath=file_path,
        )
        
        print("✓ 飞机加载成功!")
        
        # 基本信息
        print(f"飞机类型: {AC.acName}")
        print(f"BADA版本: {AC.BADAVersion}")
        print(f"BADA家族: {AC.BADAFamily}")
        print(f"BADA家族名称: {AC.BADAFamilyName}")
        
        # 基本参数
        print(f"最大起飞重量 (MTOW): {AC.MTOW:,.0f} kg")
        print(f"最大着陆重量 (MLW): {AC.MLW:,.0f} kg")
        print(f"参考面积: {AC.S:.1f} m²")
        
        # 额外参数（如果需要的话）
        try:
            print(f"空重: {AC.OEW:,.0f} kg")
            print(f"最大零燃料重量: {AC.MZFW:,.0f} kg")
        except AttributeError:
            pass  # 某些属性可能不存在
            
    except Exception as e:
        print(f"✗ 加载失败: {e}")
        continue

print("\n" + "="*80)
print("批量处理完成!")
print("="*80)

BADA 4.2 飞机参数批量获取

[ 1] 正在处理: A320-232
--------------------------------------------------
✓ 飞机加载成功!
飞机类型: A320-232
BADA版本: 4.2
BADA家族: <pyBADA.aircraft.BadaFamily object at 0x7c393e4df800>
BADA家族名称: BADA4
最大起飞重量 (MTOW): 78,000 kg
最大着陆重量 (MLW): 64,500 kg
参考面积: 122.6 m²
空重: 43,700 kg
最大零燃料重量: 61,000 kg

[ 2] 正在处理: A321-111
--------------------------------------------------
✓ 飞机加载成功!
飞机类型: A321-111
BADA版本: 4.2
BADA家族: <pyBADA.aircraft.BadaFamily object at 0x7c393e510d10>
BADA家族名称: BADA4
最大起飞重量 (MTOW): 83,000 kg
最大着陆重量 (MLW): 73,500 kg
参考面积: 122.6 m²
空重: 47,000 kg
最大零燃料重量: 69,500 kg

[ 3] 正在处理: A321-131
--------------------------------------------------
✓ 飞机加载成功!
飞机类型: A321-131
BADA版本: 4.2
BADA家族: <pyBADA.aircraft.BadaFamily object at 0x7c393e4df1d0>
BADA家族名称: BADA4
最大起飞重量 (MTOW): 83,000 kg
最大着陆重量 (MLW): 73,500 kg
参考面积: 122.6 m²
空重: 47,000 kg
最大零燃料重量: 69,500 kg

[ 4] 正在处理: A330-203
--------------------------------------------------
✓ 飞机加载成功!
飞机类型: A330-203
BADA版本: 4.2
BADA家族: <pyBADA.aircr

In [ ]:
from pyBADA.bada4 import Bada4Aircraft
import numpy as np

# 定义所有要查询的飞机型号
aircraft_types = [
    "A320-232",
    "A321-111", 
    "A321-131",
    "A330-203",
    "A330-223",
    "A330-243", 
    "A330-301",
    "A330-321",
    "A330-341",
    "A350-941",
    "A380-841",
    "A380-861",
    "B788GE70",
    "B789GE75",
    "B738W26",
    "B773RR92", 
]

# 定义巡航高度层 (FL300 到 FL410，每1000英尺一层)
flight_levels = np.arange(300, 420, 10)  # FL300, FL310, ..., FL410
altitudes_ft = flight_levels * 100  # 转换为英尺
altitudes_m = altitudes_ft * 0.3048  # 转换为米

def calculate_cruise_performance(AC, altitude_m, mass_kg=None):
    """计算指定高度的巡航性能"""
    try:
        # 如果未指定重量，使用典型巡航重量（约85%MTOW）
        if mass_kg is None:
            mass_kg = AC.MTOW * 0.85
            
        # 获取该高度的大气参数
        atm = AC.atmosphere(altitude_m)
        
        # 计算速度包络
        # 失速速度 (最小速度)
        v_stall = AC.v_stall_cr(mass_kg, altitude_m)
        
        # 最大运行速度 (VMO/MMO限制)
        v_max_op = AC.v_max_op(altitude_m)
        
        # 最大马赫数
        mach_max = getattr(AC, 'M_max', 0.82)  # 默认最大马赫数
        
        # 将马赫数转换为真空速(TAS)
        v_max_mach = mach_max * atm['a']  # a是音速
        
        # 取较小值作为实际最大速度
        v_max = min(v_max_op, v_max_mach)
        
        # 典型巡航速度(长距巡航速度LRC)
        try:
            v_cruise = AC.v_cruise_opt(mass_kg, altitude_m)
        except:
            # 如果没有最优巡航速度方法，使用经济巡航马赫数
            mach_cruise = getattr(AC, 'M_cruise', 0.78)
            v_cruise = mach_cruise * atm['a']
        
        # 计算油耗
        # 最小速度时的油耗
        try:
            fuel_flow_min = AC.fuel_flow_cruise(mass_kg, altitude_m, v_stall * 1.3)  # 1.3倍失速速度
        except:
            fuel_flow_min = 0
            
        # 巡航速度时的油耗
        try:
            fuel_flow_cruise = AC.fuel_flow_cruise(mass_kg, altitude_m, v_cruise)
        except:
            fuel_flow_cruise = 0
            
        # 最大速度时的油耗
        try:
            fuel_flow_max = AC.fuel_flow_cruise(mass_kg, altitude_m, v_max)
        except:
            fuel_flow_max = 0
        
        return {
            'v_min': v_stall * 1.3,  # 最小安全速度(1.3倍失速速度)
            'v_cruise': v_cruise,    # 巡航速度
            'v_max': v_max,          # 最大速度
            'fuel_min': fuel_flow_min,     # 最小速度油耗 (kg/s)
            'fuel_cruise': fuel_flow_cruise, # 巡航速度油耗 (kg/s)
            'fuel_max': fuel_flow_max,     # 最大速度油耗 (kg/s)
            'mach_min': (v_stall * 1.3) / atm['a'],
            'mach_cruise': v_cruise / atm['a'],
            'mach_max': v_max / atm['a']
        }
        
    except Exception as e:
        return None

# 文件路径
file_path = "/home/longqin/Downloads/4.2/BADA_4.2_L06514UPC/Models"

print("="*100)
print("BADA 4.2 飞机参数批量获取 & 巡航性能分析")
print("="*100)

# 遍历所有飞机型号
for i, ac_name in enumerate(aircraft_types, 1):
    print(f"\n[{i:2d}] 正在处理: {ac_name}")
    print("="*80)
    
    try:
        # 创建飞机对象
        AC = Bada4Aircraft(
            badaVersion="4.2",
            acName=ac_name,
            filePath=file_path,
        )
        
        print("✓ 飞机加载成功!")
        
        # ==================== 基本参数 ====================
        print("\n📊 基本参数:")
        print("-" * 40)
        print(f"飞机类型: {AC.acName}")
        print(f"BADA版本: {AC.BADAVersion}")
        print(f"BADA家族: {AC.BADAFamily}")
        print(f"BADA家族名称: {AC.BADAFamilyName}")
        print(f"最大起飞重量 (MTOW): {AC.MTOW:,.0f} kg")
        print(f"最大着陆重量 (MLW): {AC.MLW:,.0f} kg")
        print(f"参考面积: {AC.S:.1f} m²")
        
        # 额外参数
        try:
            print(f"空重: {AC.OEW:,.0f} kg")
            print(f"最大零燃料重量: {AC.MZFW:,.0f} kg")
        except AttributeError:
            pass
        
        # ==================== 巡航性能分析 ====================
        print(f"\n✈️  巡航性能分析 (FL300-FL410, 重量: {AC.MTOW*0.85:,.0f} kg)")
        print("-" * 80)
        print(f"{'FL':>4} {'Alt(ft)':>8} {'V_min':>8} {'V_cruise':>9} {'V_max':>8} "
              f"{'M_min':>7} {'M_cruise':>8} {'M_max':>7} {'FF_min':>8} {'FF_cruise':>9} {'FF_max':>8}")
        print(f"{'':>4} {'':>8} {'(kt)':>8} {'(kt)':>9} {'(kt)':>8} "
              f"{'':>7} {'':>8} {'':>7} {'(kg/h)':>8} {'(kg/h)':>9} {'(kg/h)':>8}")
        print("-" * 80)
        
        # 计算每个高度层的性能
        for j, (fl, alt_ft, alt_m) in enumerate(zip(flight_levels, altitudes_ft, altitudes_m)):
            perf = calculate_cruise_performance(AC, alt_m)
            
            if perf:
                # 转换速度单位: m/s -> knots
                v_min_kt = perf['v_min'] * 1.944  # m/s to knots
                v_cruise_kt = perf['v_cruise'] * 1.944
                v_max_kt = perf['v_max'] * 1.944
                
                # 转换油耗单位: kg/s -> kg/h
                ff_min_kgh = perf['fuel_min'] * 3600 if perf['fuel_min'] > 0 else 0
                ff_cruise_kgh = perf['fuel_cruise'] * 3600 if perf['fuel_cruise'] > 0 else 0
                ff_max_kgh = perf['fuel_max'] * 3600 if perf['fuel_max'] > 0 else 0
                
                print(f"{fl:4.0f} {alt_ft:8.0f} {v_min_kt:8.0f} {v_cruise_kt:9.0f} {v_max_kt:8.0f} "
                      f"{perf['mach_min']:7.3f} {perf['mach_cruise']:8.3f} {perf['mach_max']:7.3f} "
                      f"{ff_min_kgh:8.0f} {ff_cruise_kgh:9.0f} {ff_max_kgh:8.0f}")
            else:
                print(f"{fl:4.0f} {alt_ft:8.0f} {'---':>8} {'---':>9} {'---':>8} "
                      f"{'---':>7} {'---':>8} {'---':>7} {'---':>8} {'---':>9} {'---':>8}")
        
        print("-" * 80)
        print("注: V_min=1.3×失速速度, V_cruise=最优巡航速度, V_max=最大运营速度")
        print("    FF=燃油消耗率, M=马赫数")
        
    except Exception as e:
        print(f"✗ 加载失败: {e}")
        continue

print("\n" + "="*100)
print("批量处理完成!")
print("说明: 以上数据基于BADA 4.2模型，巡航重量假设为85%MTOW")
print("="*100)